## Global salinity budget

In [1]:
%matplotlib inline

import cosima_cookbook as cc
import matplotlib.pyplot as plt
import numpy as np
import netCDF4 as nc
import cartopy.crs as ccrs
import xarray as xr
import cmocean.cm as cmocean
import glob
import matplotlib.path as mpath
import matplotlib.colors as col

import logging
logging.captureWarnings(True)
logging.getLogger('py.warnings').setLevel(logging.ERROR)

from dask.distributed import Client


In [2]:
client = Client()
client


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 4
Total threads: 16,Total memory: 64.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:45553,Workers: 4
Dashboard: /proxy/8787/status,Total threads: 16
Started: Just now,Total memory: 64.00 GiB
Comm: tcp://127.0.0.1:39445,Total threads: 4
Dashboard: /proxy/46089/status,Memory: 16.00 GiB
Nanny: tcp://127.0.0.1:35437,


In [3]:
master_session = cc.database.create_session()

control = '01deg_jra55v13_ryf9091'

session_name = '/g/data/e14/pc5520/model_data/access-om2/basal_melt_MOM5/basal_melt_MOM5.db'
#session_name = '/g/data/v45/akm157/model_data/access-om2/basal_melt_MOM5/basal_melt_MOM5.db'
basal_melt_session = cc.database.create_session(session_name)

# temperature of meltwater using Gade line:
basal = 'accessom2-GPC015'
iceberg = 'accessom2-GPC017'


In [4]:
expt = control
session = master_session

geolon_t = cc.querying.getvar(expt,'geolon_t',session, n=1)
geolat_t = cc.querying.getvar(expt,'geolat_t',session, n=1)


In [5]:
# salinity change from restarts

# copy from heat budget notebook (Adele)

# heat content change:
# We need to calculate this from snapshots from the restarts.

salt_dummy = cc.querying.getvar(control, 'salt', master_session, n=1,frequency='1 monthly')

# note I had to collate these restarts first:
filepath = '/scratch/x77/cy8964/access-om2/archive/'+control+'/restart995/ocean/ocean_temp_salt.res.nc'
restart1t = xr.open_dataset(filepath)
# only 1 time snapshot:
salt_init = restart1t.salt[0,...]
# fix coords:
salt_init.coords['zaxis_1'] = salt_dummy['st_ocean'].values
salt_init.coords['yaxis_1'] = salt_dummy['yt_ocean'].values
salt_init.coords['xaxis_1'] = salt_dummy['xt_ocean'].values
salt_init = salt_init.rename(({'zaxis_1':'st_ocean', 'yaxis_1':'yt_ocean', 'xaxis_1':'xt_ocean'}))
salt_init = salt_init.drop_vars('Time')

filepath = '/scratch/x77/cy8964/access-om2/archive/'+control+'/restart995/ocean/ocean_thickness.res.nc'
restart1z = xr.open_dataset(filepath)
# only 1 time snapshot:
dzt_init = restart1z.rho_dzt[0,...]
# fix coords:
dzt_init.coords['zaxis_1'] = salt_dummy['st_ocean'].values
dzt_init.coords['yaxis_1'] = salt_dummy['yt_ocean'].values
dzt_init.coords['xaxis_1'] = salt_dummy['xt_ocean'].values
dzt_init = dzt_init.rename(({'zaxis_1':'st_ocean', 'yaxis_1':'yt_ocean', 'xaxis_1':'xt_ocean'}))
dzt_init = dzt_init.drop_vars('Time')

filepath = '/scratch/x77/cy8964/access-om2/archive/'+control+'/restart1035/ocean/ocean_temp_salt.res.nc'
restart2t = xr.open_dataset(filepath)
# only 1 time snapshot:
salt_end = restart2t.salt[0,...]
# fix coords:
salt_end.coords['zaxis_1'] = salt_dummy['st_ocean'].values
salt_end.coords['yaxis_1'] = salt_dummy['yt_ocean'].values
salt_end.coords['xaxis_1'] = salt_dummy['xt_ocean'].values
salt_end = salt_end.rename(({'zaxis_1':'st_ocean', 'yaxis_1':'yt_ocean', 'xaxis_1':'xt_ocean'}))
salt_end = salt_end.drop_vars('Time')

filepath = '/scratch/x77/cy8964/access-om2/archive/'+control+'/restart1035/ocean/ocean_thickness.res.nc'
restart2z = xr.open_dataset(filepath)
# only 1 time snapshot:
dzt_end = restart2z.rho_dzt[0,...]
# fix coords:
dzt_end.coords['zaxis_1'] = salt_dummy['st_ocean'].values
dzt_end.coords['yaxis_1'] = salt_dummy['yt_ocean'].values
dzt_end.coords['xaxis_1'] = salt_dummy['xt_ocean'].values
dzt_end = dzt_end.rename(({'zaxis_1':'st_ocean', 'yaxis_1':'yt_ocean', 'xaxis_1':'xt_ocean'}))
dzt_end = dzt_end.drop_vars('Time')


In [6]:
# compute salt content change:

# J / kg / degC
c_p = 3992.1
rho0 = 1035.0
area = cc.querying.getvar(control, 'area_t', master_session, n=1).load()

# there is a factor of rho_0 in dzt here:
salt_content_change = (1/rho0*((salt_end*dzt_end).sum('st_ocean') - (salt_init*dzt_init).sum('st_ocean'))*area).sum()

print('Net salt content change over this period =',salt_content_change.values/1e12,'x 10^12 g/kg m^3')

Net salt content change over this period = -0.6794126818450127 x 10^12 g/kg m^3


In [7]:
##SURFACE FLUX DURING THIS TIME
# set times to same as prev
area = cc.querying.getvar(control, 'area_t', master_session, n=1).load()

start_time = '2150-01-01'
end_time = '2159-12-31'

In [8]:
sfc_salt_flux_restore_control = cc.querying.getvar(control,'sfc_salt_flux_restore', master_session, 
                                                 start_time = start_time, end_time = end_time) 
sfc_salt_flux_restore_control = sfc_salt_flux_restore_control.sel(time = slice(start_time, end_time))
sfc_salt_flux_restore_control = (sfc_salt_flux_restore_control*sfc_salt_flux_restore_control.time.dt.days_in_month).sum('time')*24*3600 # units kg/m^2 #/sfc_salt_flux_restore_control.time.dt.days_in_month.sum('time')

sfc_salt_flux_ice_control = cc.querying.getvar(control,'sfc_salt_flux_ice', master_session, 
                                                 start_time = start_time, end_time = end_time) 
sfc_salt_flux_ice_control = sfc_salt_flux_ice_control.sel(time = slice(start_time, end_time))
sfc_salt_flux_ice_control = (sfc_salt_flux_ice_control*sfc_salt_flux_ice_control.time.dt.days_in_month).sum('time')*24*3600 # units kg/m^2 #/sfc_salt_flux_ice_control.time.dt.days_in_month.sum('time')

sfc_salt_flux_coupler_control = cc.querying.getvar(control,'sfc_salt_flux_coupler', master_session, 
                                                 start_time = start_time, end_time = end_time) 
sfc_salt_flux_coupler_control = sfc_salt_flux_coupler_control.sel(time = slice(start_time, end_time))
sfc_salt_flux_coupler_control = (sfc_salt_flux_coupler_control*sfc_salt_flux_coupler_control.time.dt.days_in_month).sum('time')*24*3600 # units kg/m^2 #/sfc_salt_flux_ice_control.time.dt.days_in_month.sum('time')


In [9]:
(area*(sfc_salt_flux_ice_control+sfc_salt_flux_restore_control+sfc_salt_flux_coupler_control)).sum().values

array(-1.40604186e+12)

In [10]:
residual = salt_content_change - (area*(sfc_salt_flux_ice_control+sfc_salt_flux_restore_control+sfc_salt_flux_coupler_control)).sum()
print(residual.values)

726629176963.2373


Salt budget = ...